In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure


path = '/Users/louis/Desktop/Tsinghua/Machine Learning/1_Project/DataSet/'
state = 'Full_Data'
full_path = path+state+'.csv'
data_set = pd.read_csv(full_path)
criteria = "Occurence"

# OR → criteria = "Percent"

# REM : f(a|b, c) means f(a) knowing b and c (e.g. with b and c fixed)

# INPUT : State & Name
# OUTPUT : Occurence = f(Year|State, Name)
def get_year(state, name, data_set_= data_set):
    return data_set_[data_set_["State"]==state][data_set_["Name"]==name]

# INPUT : Name
# OUTPUT : for each State : Occurence = f(year|State, Name)
def get_year_list(name, data_set_= data_set):
    return data_set_[data_set_["Name"]==name].pivot_table(index=["Year"],columns='State',values="Occurence")

# INPUT : Name
# OUTPUT : On average for the USA : Occurence = f(year|Name)
def get_year_USA(name, data_set_=data_set):
    if criteria=="Percent" : print("/!\ get_year_USA USES OCCURENCE, NOT PERCENT /!\ ")
    return data_set_[data_set_["Name"]==name].pivot_table(index=["Year"],columns='State',values="Occurence").sum(axis=1)

    
# INPUT : Year & Name
# OUTPUT : Occurence = f(State|Year, Name)    
def get_state(year, name, data_set_= data_set):
    return data_set_[data_set_["Year"]==year][data_set_["Name"]==name]

# INPUT : Year & State
# OUTPUT : Occurence = f(Name|Year, State)
def get_name(year, state, data_set_= data_set):
    return data_set_[data_set_["Year"]==year][data_set_["State"]==state]


# OUTPUT : List of every states in the data set
def get_list_states(data_set_ = data_set):
    return data_set["State"].drop_duplicates()


# INPUT (optional) unique = False => ith name is the most popular in the (1910+i)th year
#       (optional) period = [a,b] => only select the popular names from this period
# OUTPUT : For every year of the period, returns the most popular among the states with the number of occurences
# FORMAT : LIST
def get_list_most_used_names_year(unique = True, data_set_ = data_set):
    temp_val = data_set_.pivot_table(index="Year", columns=["Name","State"],values="Occurence").idxmax(axis=1).values
    name_list = [name for (name, state) in temp_val]
    if not unique:
        return name_list
    return [name_list[i] for i in sorted(np.unique(name_list, return_index=True)[1])]

##################################################

# >> NOT THE MOST USEFULL FUNCTIONS BELOW << 

# INPUT (optional) length = n => get the top n names. 
#       (optional) period = [a,b] => only select the popular names from this period
# OUTPUT : the most popular names among the states with the number of occurences and within the period
# FORMAT : pd.series (pandas.core.series.Series)
def get_most_used_names(length_=-1, period=(1910,2016), data_set_ = data_set):
    if criteria=="Percent" : print("/!\ get_most_used_names USES OCCURENCE, NOT PERCENT /!\ ")
    return data_set_[data_set_["Year"].isin([i for i in range(period[0],period[1]+1)])].pivot_table(index=["Year", "State"],columns='Name',values="Occurence").sum().sort_values(ascending=False)[0:length_]

# SAME AS "get_most_used_names()" but RETURNS a LIST with only the names
# FORMAT : LIST
def get_list_most_used_names(length_=-1, period=(1910,2016), data_set_ = data_set):
    return list(get_most_used_names(length_, period, data_set_).index)



In [2]:
nbr_states = 51
# hyperparameters
lr = 0.01

n_iter_training = 500
batch_size=1

n_inputs = 1   # En input on a un scalaire (qu'on regarde plusieurs fois #n_steps)
length_data = 110 # Nombre d'années qu'on a

n_steps = 50    # Années qu'on regarde
n_iter_steps = length_data - n_steps-1
n_hidden_units = 10   # A voir
n_classes = 1      # Regression !

In [3]:
# list_values = get_year_list("Mary").values
import sklearn
from sklearn.preprocessing import normalize

def get_xy(list_values_, n_steps_):
    x_ = np.empty((len(list_values_) - n_steps_-1, n_steps_, n_inputs), int)
    y_ = np.empty((len(list_values_) - n_steps_-1, 1), int)
    
    for i in range(len(list_values_) - n_steps_-1):
        x_[i] = list_values[i:i+n_steps_]
        y_[i] = list_values[i+n_steps_][0] # On étudie le premier état

    return x_, y_
        
    
list_values = np.array([[j for i in range(n_inputs)] for j in range(length_data)])
x_values, y_values = get_xy(list_values, n_steps)

NameError: name 'np' is not defined

## TF

In [610]:
import tensorflow as tf
tf.reset_default_graph()


# tf Graph input
x = tf.placeholder(tf.float32, [batch_size, n_steps, n_inputs], name="x_placeholder")
y = tf.placeholder(tf.float32, [batch_size, n_classes], name="y_placeholder")

# Define weights
weights = {
    'in': tf.Variable(tf.truncated_normal([n_inputs, n_hidden_units],
                                    stddev=1./10)),
    'out': tf.Variable(tf.truncated_normal([n_hidden_units, n_classes],
                                    stddev=1./10))
}
biases = {
    'in': tf.Variable(tf.constant(0.1, shape=[n_hidden_units, ])),
    'out': tf.Variable(tf.constant(0.1, shape=[n_classes, ]))
}

In [611]:
np.shape(x_values)

(59, 50, 1)

In [612]:
def RNN(X, weights, biases):


    # X ==> ( batch * steps, inputs)
    X = tf.reshape(X, [-1, n_inputs])
    
    # X_in = ( batch, steps, hidden)
    X_in = tf.matmul(X, weights['in']) + biases['in']

    # X_in ==> ( batch, steps, hidden)
    X_in = tf.reshape(X_in, [-1, n_steps, n_hidden_units])
    

    # basic LSTM Cell.
    cell = tf.nn.rnn_cell.LSTMCell(n_hidden_units)
    init_state = cell.zero_state(batch_size, dtype=tf.float32)

    outputs, final_state = tf.nn.dynamic_rnn(cell, X_in, initial_state=init_state, time_major=False)

    #Encore à regarder ces 2 lignes !
    #outputs = tf.unstack(tf.transpose(outputs, [1,0,2]))
    results = tf.matmul(outputs[:, -1], weights['out']) + biases['out'] 
    print("OUT",outputs)
    return results, outputs[:, -1]

In [613]:
pred, outputs = RNN(x, weights, biases)
# cost = tf.reduce_mean(tf.squared_difference(tf.stack(pred), tf.stack(y)))
cost = tf.losses.mean_squared_error(predictions=pred, labels=y)
train_op = tf.train.AdamOptimizer(lr).minimize(cost)

# correct_pred = tf.stack([tf.argmax(pred, 1), tf.argmax(y, 1)])
# accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# cost = tf.reduce_mean(tf.squared_difference(tf.stack(pred), tf.stack(y)))
#temp_output = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=pred, labels=tf.cast(tf.reshape(y, [1]), tf.int64))

OUT Tensor("rnn/transpose_1:0", shape=(1, 50, 10), dtype=float32)


In [614]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    #TRAINING PHASE
    for i in range(n_iter_training):
        print("###########################")
        print(i)
        print("###########################")
        for j in range(n_iter_steps):
            batch_xs = x_values[j].reshape(-1,n_steps, n_inputs)
            batch_ys = y_values[j].reshape(-1,1)

            sess.run([train_op], feed_dict={
                x: batch_xs,
                y: batch_ys,
            })



            if (i%20==0 and j%10==0):
                print("")
                print(batch_xs)
                print("OUT = ",sess.run(outputs, feed_dict={
                    x: batch_xs,
                    y: batch_ys,
                }))
                print("biases['out'] = ",sess.run( biases['out'], feed_dict={
                    x: batch_xs,
                    y: batch_ys,
                })) 
                print("pred = ",sess.run(pred, feed_dict={
                    x: batch_xs,
                    y: batch_ys,
                }))

                print("y = ",sess.run(y, feed_dict={
                    x: batch_xs,
                    y: batch_ys,
                }))



#                 print("cost = ",sess.run(cost, feed_dict={
#                     x: batch_xs,
#                     y: batch_ys,
#                 }))
    #         print("TRAINING DONE")
    #         print("cost = ",sess.run(cost, feed_dict={
    #             x: batch_xs,
    #             y: batch_ys,
    #         }))

    
    # PREDICTION PHASE

###########################
0
###########################

[[[ 0]
  [ 1]
  [ 2]
  [ 3]
  [ 4]
  [ 5]
  [ 6]
  [ 7]
  [ 8]
  [ 9]
  [10]
  [11]
  [12]
  [13]
  [14]
  [15]
  [16]
  [17]
  [18]
  [19]
  [20]
  [21]
  [22]
  [23]
  [24]
  [25]
  [26]
  [27]
  [28]
  [29]
  [30]
  [31]
  [32]
  [33]
  [34]
  [35]
  [36]
  [37]
  [38]
  [39]
  [40]
  [41]
  [42]
  [43]
  [44]
  [45]
  [46]
  [47]
  [48]
  [49]]]
OUT =  [[ 0.03591835 -0.01068304 -0.99687785  0.06227979  0.34952787  0.25995284
  -0.31606853 -0.6233327  -0.05133756  0.09816837]]
biases['out'] =  [0.11]
pred =  [[0.31693095]]
y =  [[50.]]

[[[10]
  [11]
  [12]
  [13]
  [14]
  [15]
  [16]
  [17]
  [18]
  [19]
  [20]
  [21]
  [22]
  [23]
  [24]
  [25]
  [26]
  [27]
  [28]
  [29]
  [30]
  [31]
  [32]
  [33]
  [34]
  [35]
  [36]
  [37]
  [38]
  [39]
  [40]
  [41]
  [42]
  [43]
  [44]
  [45]
  [46]
  [47]
  [48]
  [49]
  [50]
  [51]
  [52]
  [53]
  [54]
  [55]
  [56]
  [57]
  [58]
  [59]]]
OUT =  [[-3.1533396e-05  2.9667013e-04 -9.9

KeyboardInterrupt: 